<a href="https://colab.research.google.com/github/ChinmayNakwa/Research_Papers/blob/main/Deep_Residual_Learning_for_Image_Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Sample Dataset

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("gpiosenka/cards-image-datasetclassification")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/cards-image-datasetclassification


In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    print(dirname)

/kaggle/input
/kaggle/input/cards-image-datasetclassification
/kaggle/input/cards-image-datasetclassification/valid
/kaggle/input/cards-image-datasetclassification/valid/ace of spades
/kaggle/input/cards-image-datasetclassification/valid/seven of clubs
/kaggle/input/cards-image-datasetclassification/valid/jack of clubs
/kaggle/input/cards-image-datasetclassification/valid/two of clubs
/kaggle/input/cards-image-datasetclassification/valid/queen of hearts
/kaggle/input/cards-image-datasetclassification/valid/two of hearts
/kaggle/input/cards-image-datasetclassification/valid/four of diamonds
/kaggle/input/cards-image-datasetclassification/valid/queen of clubs
/kaggle/input/cards-image-datasetclassification/valid/ten of spades
/kaggle/input/cards-image-datasetclassification/valid/eight of diamonds
/kaggle/input/cards-image-datasetclassification/valid/three of clubs
/kaggle/input/cards-image-datasetclassification/valid/eight of hearts
/kaggle/input/cards-image-datasetclassification/valid/t

In [ ]:
train_path = "/kaggle/input/cards-image-datasetclassification/train/"
test_path = "/kaggle/input/cards-image-datasetclassification/test/"
val_path = "/kaggle/input/cards-image-datasetclassification/valid/"

In [ ]:
import os
print(os.path.exists(train_path))
print(os.path.exists(test_path))
print(os.path.exists(val_path))

True
True
True


## Resnet

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers

In [ ]:
def identity_block(x, filters):
  x_shortcut = x
  x = layers.Conv2D(filters, (3, 3), padding='same')(x)
  x = layers.BatchNormailization()(x)
  x = layers.Activation('relu')(x)
  x = layers.Conv2D(filters, (3, 3), padding='same')(x)
  x = layers.BatchNormailzation()(x)
  x = layers.Add()([x, x_shortcut])
  x = layers.Activation('relu')(x)
  return x

In [ ]:
def convolutional_block(x, filters, s):
  x_shortcut = x
  x = layers.Conv2D(filters, (3, 3), padding='same', strides=(2, 2))(x)
  x = layers.BatchNormalization()(x)
  x = layers.Activation('relu')(x)
  x = layers.Conv2D(filters, (3, 3), padding='same')(x)
  x = layers.BatchNormalization()(x)
  x_shortcut = layers.Conv2D(filters, (1, 1), strides=(2, 2))(x_shortcut)
  x_shortcut = layers.BatchNormalization()(x_shortcut)
  x = layers.Add()([x, x_shortcut])
  x = layers.Activation('relu')(x)
  return x

In [ ]:
def bottleneck_identity_block(x, f, filters):
  x_shortcut = x
  f1, f2, f3 = filters
  x = layers.Conv2D(f1, kernel_size=(1, 1))(x)
  x = layers.BatchNormalization(axis=3)(x)
  x = layers.Activation('relu')(x)
  x = layers.Conv2D(f2, kernel_size=(f, f), padding='same')(x)
  x = layers.BatchNormalization(axis=3)(x)
  x = layers.Activation('relu')(x)
  x = layers.Conv2D(f3, kernel_size=(1, 1))(x)
  x = layers.BatchNormalization(axis=3)(x)
  x = layers.Add()([x, x_shortcut])
  x = layers.Activation('relu')(x)
  return x

In [ ]:
def bottleneck_convolutional_block(x, f, filters, s = 2):
  x_shortcut = x
  f1, f2, f3 = filters
  x = layers.Conv2D(filters=f1, kernel_size=(1, 1), strides=(s, s))(x)
  x = layers.BatchNormalization()(x)
  x = layers.Activation('relu')(x)
  x = layers.Conv2D(filters=f2, kernel_size=(f, f), padding='same')(x)
  x = layers.BatchNormalization()(x)
  x = layers.Activation('relu')(x)
  x = layers.Conv2D(filters=f3, kernel_size=(1, 1))(x)
  x = layers.BatchNormalization()(x)
  x_shortcut = layers.Conv2D(filters=f3, kernel_size=(1, 1), strides=(s, s))(x_shortcut)
  x_shortcut = layers.BatchNormalization(axis=3)(x_shortcut)
  x = layers.Add()([x, x_shortcut])
  x = layers.Activation('relu')(x)
  return x

In [ ]:
def ResNet50(input_shape=(224, 224, 3), classes = 5):
  X_input = layers.Input(input_shape)

  # Stage 1
  x = layers.Conv2D(64, (7, 7), strides=(2, 2), padding='same')(X_input)
  x = layers.BatchNormalization(axis=3)(x)
  x = layers.Activation('relu')(x)
  x = layers.MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)

  # Stage 2
  x = bottleneck_convolutional_block(x, f=3, filters=[64, 64, 256], s = 1)
  x = bottleneck_identity_block(x, 3, [64, 64, 256])
  x = bottleneck_identity_block(x, 3, [64, 64, 256])

  # Stage 3
  x = bottleneck_convolutional_block(x, f=3, filters=[128, 128, 512], s = 2)
  x = bottleneck_identity_block(x, 3, [128, 128, 512])
  x = bottleneck_identity_block(x, 3, [128, 128, 512])
  x = bottleneck_identity_block(x, 3, [128, 128, 512])

  # Stage 4
  x = bottleneck_convolutional_block(x, f=3, filters=[256, 256, 1024], s = 2)
  x = bottleneck_identity_block(x, 3, [256, 256, 1024])
  x = bottleneck_identity_block(x, 3, [256, 256, 1024])
  x = bottleneck_identity_block(x, 3, [256, 256, 1024])
  x = bottleneck_identity_block(x, 3, [256, 256, 1024])
  x = bottleneck_identity_block(x, 3, [256, 256, 1024])

  # Stage 5
  x = bottleneck_convolutional_block(x, f=3, filters=[512, 512, 2048], s=2)
  x = bottleneck_identity_block(x, 3, [512, 512, 2048])
  x = bottleneck_identity_block(x, 3, [512, 512, 2048])

  # Stage 6
  x = layers. GlobalAveragePooling2D()(x)
  x = layers.Dense(classes, activation='softmax')(x)

  model = tf.keras.Model(inputs=X_input, outputs=x)
  return model

In [ ]:
resnet_50 = ResNet50()

In [ ]:
resnet_50.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_53 (Conv2D)  │ (None, 112, 112,  │      9,472 │ input_layer_1[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 112, 112,  │        256 │ conv2d_53[0][0]   │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_49       │ (None, 112, 112,  │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_1     │ (None, 56, 56,    │          0 │ activation_49[0]… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_54 (Conv2D)  │ (None, 56, 56,    │      4,160 │ max_pooling2d_1[… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 56, 56,    │        256 │ conv2d_54[0][0]   │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_50       │ (None, 56, 56,    │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_55 (Conv2D)  │ (None, 56, 56,    │     36,928 │ activation_50[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 56, 56,    │        256 │ conv2d_55[0][0]   │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_51       │ (None, 56, 56,    │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_56 (Conv2D)  │ (None, 56, 56,    │     16,640 │ activation_51[0]… │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_57 (Conv2D)  │ (None, 56, 56,    │     16,640 │ max_pooling2d_1[… │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 56, 56,    │      1,024 │ conv2d_56[0][0]   │
│ (BatchNormalizatio… │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 56, 56,    │      1,024 │ conv2d_57[0][0]   │
│ (BatchNormalizatio… │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_16 (Add)        │ (None, 56, 56,    │          0 │ batch_normalizat… │
│                     │ 256)              │            │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_52       │ (None, 56, 56,    │          0 │ add_16[0][0]    

 Total params: 23,597,957 (90.02 MB)

 Trainable params: 23,544,837 (89.82 MB)

 Non-trainable params: 53,120 (207.50 KB)

In [ ]:
IMG_HEIGHT = 224
IMG_WIDTH = 224
BATCH_SIZE = 32

In [ ]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    train_path,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    label_mode='categorical',
    shuffle=True
)

test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    test_path,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    label_mode='categorical',
    shuffle=False
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    val_path,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    label_mode='categorical',
    shuffle=False
)

Found 7624 files belonging to 53 classes.
Found 265 files belonging to 53 classes.
Found 265 files belonging to 53 classes.


In [ ]:
data_augmentation = tf.keras.models.Sequential(
    [
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.2),
        layers.RandomZoom(0.2),
        layers.RandomHeight(0.2)
    ],
    name="data_augmentation",
)

In [ ]:
def rescale_pixels(image, label):
  return image / 255.0, label

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

In [ ]:
train_ds = train_ds.map(rescale_pixels, num_parallel_calls=AUTOTUNE)
train_ds = train_ds.map(lambda x, y: (data_augmentation(x, training=True), y), num_parallel_calls=AUTOTUNE)
train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)

val_ds = val_ds.map(rescale_pixels, num_parallel_calls=AUTOTUNE).cache().prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.map(rescale_pixels, num_parallel_calls=AUTOTUNE).cache().prefetch(buffer_size=AUTOTUNE)


In [ ]:
model = ResNet50(input_shape=(IMG_HEIGHT, IMG_WIDTH, 3), classes=53)

In [ ]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
model.summary()

Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_5       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_159 (Conv2D) │ (None, 112, 112,  │      9,472 │ input_layer_5[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 112, 112,  │        256 │ conv2d_159[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_147      │ (None, 112, 112,  │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_3     │ (None, 56, 56,    │          0 │ activation_147[0… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_160 (Conv2D) │ (None, 56, 56,    │      4,160 │ max_pooling2d_3[… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 56, 56,    │        256 │ conv2d_160[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_148      │ (None, 56, 56,    │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_161 (Conv2D) │ (None, 56, 56,    │     36,928 │ activation_148[0… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 56, 56,    │        256 │ conv2d_161[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_149      │ (None, 56, 56,    │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_162 (Conv2D) │ (None, 56, 56,    │     16,640 │ activation_149[0… │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_163 (Conv2D) │ (None, 56, 56,    │     16,640 │ max_pooling2d_3[… │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 56, 56,    │      1,024 │ conv2d_162[0][0]  │
│ (BatchNormalizatio… │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 56, 56,    │      1,024 │ conv2d_163[0][0]  │
│ (BatchNormalizatio… │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_48 (Add)        │ (None, 56, 56,    │          0 │ batch_normalizat… │
│                     │ 256)              │            │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_150      │ (None, 56, 56,    │          0 │ add_48[0][0]    

 Total params: 23,696,309 (90.39 MB)

 Trainable params: 23,643,189 (90.19 MB)

 Non-trainable params: 53,120 (207.50 KB)

In [ ]:
EPOCHS = 25

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS
)

Epoch 1/25
  7/239 ━━━━━━━━━━━━━━━━━━━━ 1:28:59 23s/step - accuracy: 0.0234 - loss: 6.9833

In [ ]:
print("\n--- Evaluating Model on Test Data ---")
results = model.evaluate(test_ds)
print(f"Test Loss: {results[0]:.4f}")
print(f"Test Accuracy: {results[1]:.4f}")